# i.MX RT118x External Memory Configuration with SPSDK

#### This notebook shows how an external memory can be configured using the SPSDK. All of the commands executed in the interactive cells can be run from a normal CMD or PowerShell as well.

## 1. Prepare the environment

#### Run the code below to initialize parametrs for the script:

In [1]:
%run ../init_notebook.ipynb

import os
import pprint

from IPython.display import Code

pp = pprint.PrettyPrinter(indent=4)

WORKSPACE = "workspace/"  # change this to path to your workspace
INPUTS = "inputs/"
VERBOSITY = (
    "-v"  # verbosity of commands, might be -v or -vv for debug or blank for no additional info
)

FLASHLOADER_FILE = "../flashloader/ahab/workspace/flashloader.bin"  # Path to Flashloader
CFG_MEM_FILE = WORKSPACE + "config_mem.bls"  # Configure memory template
CUSTOM_CFG_MEM_FILE = WORKSPACE + "custom_config_mem.bls"  # Configure memory template
FCB_FILE = WORKSPACE + "fcb.bin"  # Firmware Configuration block file
CUSTOM_FCB_FILE = WORKSPACE + "custom_fcb.bin"  # Firmware Configuration block file

env: JUPYTER_SPSDK=1
Created `%!` as an alias for `%execute`.


#### Generate a flashloader binary
We will need program our boards with a flashloader binary to be able to send blhost commands to it. To get such a binary, we can use the [signed_flashloader notebook](../flashloader/ahab/rt118x_signed_flashloader.ipynb). The details are out of scope for this session, so just open the [signed_flashloader notebook](../flashloader/ahab/rt118x_signed_flashloader.ipynb), use the "Run All Cells" option and close it again.

![Run All Cells](./img/run_all_cells.png "run all cells")

## 2. Connect your board to you PC. Use both the Debug USB Port and USB OTG1 port and set the board to SDP mode.

### EVK Board Overview

Configure Boot Mode Switch to SDP: 1-OFF, 2-OFF, 3-OFF, 4-ON\
Connect your board to your PC via a micro USB using the USB OTG1 port - J33\
Connect your board to your PC via a micro USB using the Debug USB port - J53

#### *The following picture describes connector placement of RT1180 EVK:*
![Signing key and used SRK definition](img/evk-1.png "signing key")

## 3. Scan for available devices with **nxpdevscan**, connect to the i.MX RT1180 and load a flashloader into it using **blhost**

#### Call **nxpdevscan --help** to get a list of options supported by **nxpdevscan**:

In [2]:
%! nxpdevscan --help

nxpdevscan --help 
Usage: nxpdevscan [OPTIONS]

  Utility listing all connected NXP USB and UART devices.

  NOTE: This utility lists all NXPs USB and UART devices connected to the
  host. By default it scans UART devices by pinging them (sending the mboot or
  SDP command). This however causes that the device ISP mode is locked to
  UART. Use the -n/--no-scan option to disable this behavior. If you want to
  only scan for NXP UART devices, use the --nxp option.

Options:
  -e, --extend-vids TEXT          VID in hex to extend search.
  -o, --output FILENAME
  -n, --no-scan                   Do not scan UART devices by pinging them.
  --nxp                           Scan only NXP UART devices.
  Narrow down the scope of scanning: [mutually_exclusive]
    -a, --all                     Search for all NXP devices (default)
    -u, --usb                     Search only for USB devices
    -sd, --sdio                   Search only for SDIO devices
    -p, --port                    Search onl

In [3]:
%! nxpdevscan

nxpdevscan 
-------- Connected NXP USB Devices --------

MCU-LINK NXP TRACE - NXP Semiconductors
Vendor ID: 0x1fc9
Product ID: 0x0143
Path: HID\VID_1FC9&PID_0143&MI_01\A&BEC7DF5&0&0000
Path Hash: 884d05f8
Name: k32w1xx | kw45xx | mcxw71xx
Serial number: 5RR4XWAAQLSBP

USB COMPOSITE DEVICE - FREESCALE SEMICONDUCTOR INC.
Vendor ID: 0x15a2
Product ID: 0x0073
Path: HID\VID_15A2&PID_0073\A&B33C6E3&0&0000
Path Hash: 29fa2649
Name: rt118x | rt1010 | rt102x | mwct2xxxs | rt1015 | rt105x | rt106x | rt104x | rt117x | rt116x
Serial number: 

-------- Connected NXP UART Devices --------

Port: COM13
Type: mboot device

-------- Connected NXP SIO Devices --------



#### The Vendor and Product ID values of the **OO Blank RT Family - NXP SEMICONDUCTORS** are the default values for the boot ROM.

#### Store them in a variable for later use:

In [5]:
COMPAR = "-u"  # USB
BLHOST_CONNECT_ROM = "0x1fc9,0x014c"  # PID/VID of ROM

&nbsp;
### Reset your board by pressing the **SW3** button. This is necessary because calling **nxpdevscan** causes the boot ROM to stop being visible. You can test this by executing the **nxpdevscan** cell one more time.
&nbsp;

#### Ping the boot ROM via **blhost**:

In [6]:
%! blhost $COMPAR $BLHOST_CONNECT_ROM get-property 1

blhost -u 0x1fc9,0x014c get-property 1 
Response status = 0 (0x0) Success.
Response word 1 = 1258487809 (0x4b030001)
Current Version = K3.0.1


#### Load a flashloader image into the device:

In [7]:
from time import sleep

assert os.path.exists(FLASHLOADER_FILE)

%! blhost $COMPAR $BLHOST_CONNECT_ROM load-image $FLASHLOADER_FILE
sleep(3)  # wait for device

blhost -u 0x1fc9,0x014c load-image ../flashloader/ahab/workspace/flashloader.bin 
Loading image
Response status = 0 (0x0) Success.


#### Call **nxpdevscan** again to get the Value and Product ID of the flashloader:

In [8]:
%! nxpdevscan

nxpdevscan 
-------- Connected NXP USB Devices --------

MCU-LINK NXP TRACE - NXP Semiconductors
Vendor ID: 0x1fc9
Product ID: 0x0143
Path: HID\VID_1FC9&PID_0143&MI_01\A&BEC7DF5&0&0000
Path Hash: 884d05f8
Name: mcxw71xx | k32w1xx | kw45xx
Serial number: 5RR4XWAAQLSBP

USB COMPOSITE DEVICE - FREESCALE SEMICONDUCTOR INC.
Vendor ID: 0x15a2
Product ID: 0x0073
Path: HID\VID_15A2&PID_0073\A&B33C6E3&0&0000
Path Hash: 29fa2649
Name: rt102x | rt118x | rt106x | rt117x | rt105x | rt104x | rt1015 | rt116x | mwct2xxxs | rt1010
Serial number: 

-------- Connected NXP UART Devices --------

Port: COM13
Type: mboot device

-------- Connected NXP SIO Devices --------



#### Store the values in a variable again:

In [9]:
BLHOST_CONNECT_FLDR = "0x15A2,0x0073"  # PID/VID of NXP Flashloader

&nbsp;
### Reset your board by pressing the **SW3** button. This is necessary because calling **nxpdevscan** causes the flashloader to stop being visible as well. You can test this by executing the **nxpdevscan** cell one more time.
&nbsp;

#### **Reload the flashloader** to be able to continue:

In [10]:
%! blhost $COMPAR $BLHOST_CONNECT_ROM load-image $FLASHLOADER_FILE
sleep(3)  # wait for device

blhost -u 0x1fc9,0x014c load-image ../flashloader/ahab/workspace/flashloader.bin 
Loading image
Response status = 0 (0x0) Success.


#### Ping the flashloader to double check everything worked correctly:

In [11]:
%! blhost $VERBOSITY $COMPAR $BLHOST_CONNECT_FLDR get-property 1

blhost -v -u 0x15A2,0x0073 get-property 1 
INFO:spsdk.mboot.mcuboot:Connect: identifier='usb', device=USB COMPOSITE DEVICE (0x15A2, 0x0073)path=b'\\\\?\\hid#vid_15a2&pid_0073#a&b33c6e3&0&0000#{4d1e55b2-f16f-11cf-88cb-001111000030}' sn=''
INFO:spsdk.mboot.mcuboot:CMD: GetProperty(CurrentVersion, index=0)
INFO:spsdk.mboot.mcuboot:CMD: Status: 0 (0x0) Success.
Response status = 0 (0x0) Success.
Response word 1 = 1258424320 (0x4b020800)
Current Version = K2.8.0
INFO:spsdk.mboot.mcuboot:Closing: identifier='usb', device=USB COMPOSITE DEVICE (0x15A2, 0x0073)path=b'\\\\?\\hid#vid_15a2&pid_0073#a&b33c6e3&0&0000#{4d1e55b2-f16f-11cf-88cb-001111000030}' sn=''


#### Check the enabled flashloader commands:

In [12]:
%! blhost $COMPAR $BLHOST_CONNECT_FLDR get-property 7

blhost -u 0x15A2,0x0073 get-property 7 
Response status = 0 (0x0) Success.
Response word 1 = 25530335 (0x1858fdf)
Available Commands = ['FlashEraseAll', 'FlashEraseRegion', 'ReadMemory', 'WriteMemory', 'FillMemory', 'GetProperty', 'ReceiveSBFile', 'Execute', 'Call', 'Reset', 'SetProperty', 'FlashReadResource', 'ConfigureMemory', 'GenerateKeyBlob', 'UpdateLifeCycle', 'EleMessage']


## 4. Generate a blhost script for memory configuration using **nxpmemcfg**

#### Call **nxpmemcfg** or **nxpmemcfg --help** to get a list of options and sub-commands supported by **nxpmemcfg**

In [13]:
%! nxpmemcfg

nxpmemcfg 
Usage: nxpmemcfg [OPTIONS] COMMAND [ARGS]...

  Collection of utilities for memory configuration operations.

Options:
  -v, --verbose  Print more detailed information
  -vv, --debug   Display more debugging information.
  --version      Show the version and exit.
  --help         Show this message and exit.

Commands:
  nxpmemcfg          Collection of utilities for memory configuration operations.
  ├── blhost-script  Export the configuration option words to blhost script.
  ├── export         Export the configuration option words from configuration.
  ├── family-info    List known memory configurations for the family.
  ├── get-templates  Create template of Memory option words in YAML format.
  └── parse          Parse the existing memory configuration option words.


#### Call **nxpmemcfg blhost-script** or **nxpmemcfg blhost-script --help** to get a list of options supported by **nxpmemcfg blhost-script**:

In [14]:
%! nxpmemcfg blhost-script

nxpmemcfg blhost-script 
Usage: nxpmemcfg blhost-script [OPTIONS]

  Export the configuration option words to blhost script.

Options:
  Known chip select. Alternative to known chip is YAML configuration '-c': [all_or_none]
    -f, --family [lpc5502|lpc5504|lpc5506|lpc5512..., and more. Use 'get-families' command to show all.]
                                  Select the chip family.
    -m, --memory-chip TEXT        Select supported memory chip name. Use
                                  family-info command to get the known names.
    -i, --interface TEXT          Select supported memory chip interface. Use
                                  family-info command to get the known
                                  interfaces.
  -c, --config FILE               Option word configuration YAML file, in case
                                  that known chip has not been used
  -ix, --instance INTEGER         Instance of peripheral if applicable
  --fcb PATH                      Optional filena

#### As you can see, there are several parameters we will need to be able to generate the blhost-script:
- f: device family, we will choose mimxrt1189
- m: memory chip, we need to find out the correct name
- i: memory interface, we need to find out the correct name
- ix: peripheral instance index, we need to find out the correct value
- o and fcb output file paths are up to our choice. This was configured in the first step into the CFG_MEM_FILE and FCB_FILE variables.

#### Call **nxpmemcfg family-info** to check the known memories and their default configurations:

In [15]:
%! nxpmemcfg family-info

nxpmemcfg family-info 
List of all supported peripherals and its instances:
╔════╦════════════╦═════════════╦══════════════╦══════════╦═══════════╦══════════════════════════╦════════╦════════╗
║ #  ║   Family   ║ flexspi_nor ║ flexspi_nand ║ semc_nor ║ semc_nand ║         spi_nor          ║  mmc   ║   sd   ║
╠════╬════════════╬═════════════╬══════════════╬══════════╬═══════════╬══════════════════════════╬════════╬════════╣
║ 0  ║  lpc5502   ║     N/A     ║     N/A      ║   N/A    ║    N/A    ║           [3]            ║  N/A   ║  N/A   ║
║ 1  ║  lpc5504   ║     N/A     ║     N/A      ║   N/A    ║    N/A    ║           [3]            ║  N/A   ║  N/A   ║
║ 2  ║  lpc5506   ║     N/A     ║     N/A      ║   N/A    ║    N/A    ║           [3]            ║  N/A   ║  N/A   ║
║ 3  ║  lpc5512   ║     N/A     ║     N/A      ║   N/A    ║    N/A    ║           [3]            ║  N/A   ║  N/A   ║
║ 4  ║  lpc5514   ║     N/A     ║     N/A      ║   N/A    ║    N/A    ║           [3]            ║  N/A  

#### Since we're using the EVK, we are interested in the default FlexSPI NOR from Winbond on interface index 1:

#### Note that if you use this approach, you will configure your memory with the default option words shown in the table. You will see how to change the default parameters in chapter 6.

#### Now we can set the options according to the table:
- f: device family => **mimxrt1189**
- m: memory chip => **W25QxxxJV**
- i: memory interface => **quad_spi**
- ix: peripheral instance index => **1**

In [16]:
%! nxpmemcfg blhost-script -f mimxrt1189 -m W25QxxxJV -i quad_spi -ix 1 -o $CFG_MEM_FILE --force --fcb $FCB_FILE

assert os.path.exists(CFG_MEM_FILE)

nxpmemcfg blhost-script -f mimxrt1189 -m W25QxxxJV -i quad_spi -ix 1 -o workspace/config_mem.bls --force --fcb workspace/fcb.bin 
Detected peripheral flexspi_nor for W25QxxxJV
Loaded option words: 0xC0000207
Exported blhost script.


#### Let's look inside the blhost-script:

In [17]:
Code(filename=CFG_MEM_FILE, language="bash")

# BLHOST configure memory programming script
# Generated by SPSDK NXPMEMCFG tool
# Chip: mimxrt1189
# Peripheral: flexspi_nor
# Instance: 1

# Switch the instance of the peripheral to 1:
fill-memory 0x1FFE0000 4 0xCF900001
configure-memory 9 0x1FFE0000

# Configure memory:
# Option word 0: 0xC0000207
fill-memory 0x1FFE0000 4 0xC0000207
configure-memory 9 0x1FFE0000

# Script to erase FCB location, create FCB and read back a FCB block:
flash-erase-region 0x28000000 0x1000
fill-memory 0x1FFE0000 4 0xF000000F
configure-memory 9 0x1FFE0000
read-memory 0x28000400 0x200 workspace/fcb.bin

## 5. Execute the blhost script

In [18]:
# The board must be in SDP mode
# Configure Boot Mode Switch to : 1/2/3-OFF, 4-ON
# Connect micro USB cable into USB OTG1         - J33
# Connect micro USB cable into Debug USB Port   - J53

# FLASHLOADER_FILE = "../flashloader/ahab/workspace/flashloader.bin"  # Path to Flashloader
# CFG_MEM_FILE = WORKSPACE + "config_mem.bls"                         # Configure memory template
# FCB_FILE = WORKSPACE + "fcb.bin"                                    # Firmware Configuration block file

# COMPAR = "-u"                          # USB
# BLHOST_CONNECT_ROM = "0x1fc9,0x014c"   # PID/VID of ROM
# BLHOST_CONNECT_FLDR = "0x15A2,0x0073"  # PID/VID of NXP Flashloader

# Execute the commands generated by "blhost-script". This script will configure flash,
# program FCB into the external memory and reads back the binary data. Be aware tha erase of the first sector must be done.
# The script can be executed only once to read FCB.
%! blhost $COMPAR $BLHOST_CONNECT_FLDR batch $CFG_MEM_FILE

blhost -u 0x15A2,0x0073 batch workspace/config_mem.bls 
Response status = 0 (0x0) Success.
Response status = 0 (0x0) Success.
Response status = 0 (0x0) Success.
Response status = 0 (0x0) Success.
Response status = 0 (0x0) Success.
Response status = 0 (0x0) Success.
Response status = 0 (0x0) Success.
Reading memory
Response status = 0 (0x0) Success.
Response word 1 = 512 (0x200)
Read 512 of 512 bytes.


## 6. Create a non-default memory configuration

#### The **nxpmemcfg get-templates** sub-command can be used to generate yaml configuration templates that can be modified as needed:

In [19]:
%! nxpmemcfg get-templates

nxpmemcfg get-templates 
Usage: nxpmemcfg get-templates [OPTIONS]

  Create template of Memory option words in YAML format.

Options:
  -f, --family [lpc5502|lpc5504|lpc5506|lpc5512..., and more. Use 'get-families' command to show all.]
                                  Select the chip family.
  -o, --output DIRECTORY          Path to a directory, where to store
                                  generated/parsed files.  [required]
  --force                         Force overwriting of existing files.
  --help                          Show this message and exit.


In [20]:
%! nxpmemcfg get-templates -f mimxrt1189 -o workspace/cfg_opt_templates/ --force

nxpmemcfg get-templates -f mimxrt1189 -o workspace/cfg_opt_templates/ --force 
The Memory Configuration template for mimxrt1189 has been saved into workspace/cfg_opt_templates YAML file
The Memory Configuration template for mimxrt1189 has been saved into workspace/cfg_opt_templates YAML file
The Memory Configuration template for mimxrt1189 has been saved into workspace/cfg_opt_templates YAML file
The Memory Configuration template for mimxrt1189 has been saved into workspace/cfg_opt_templates YAML file
The Memory Configuration template for mimxrt1189 has been saved into workspace/cfg_opt_templates YAML file
The Memory Configuration template for mimxrt1189 has been saved into workspace/cfg_opt_templates YAML file
The Memory Configuration template for mimxrt1189 has been saved into workspace/cfg_opt_templates YAML file


#### Let's inspect the **flexspi_nor** template:

In [21]:
Code(filename="workspace/cfg_opt_templates/ow_flexspi_nor.yaml", language="yaml")

# =========================  Option Words Configuration template for mimxrt1189, flexspi_nor.  =========================


# -------------------------------------------------------- Note --------------------------------------------------------
# Note for settings:
# All registers is possible to define also as one value although the bitfields are used. Instead of bitfields: ...
# field, the value: ... definition works as well.
# ======================================================================================================================
#                                                 == General Options ==                                                 
# ======================================================================================================================
# -------------------------------------===== The chip family name [Required] =====--------------------------------------
# Description: NXP chip family identifier.
# Possible options: <lpc5502, lpc5504, lpc5506, lpc5512, lpc5514, lpc5516, lpc5526, lpc5528, lpc55s04, lpc55s06,
# lpc55s14, lpc55s16, lpc55s26, lpc55s28, lpc55s36, lpc55s66, lpc55s69, mcxn235, mcxn236, mimxrt1010, mimxrt1015,
# mimxrt1020, mimxrt1024, mimxrt1040, mimxrt1050, mimxrt1060, mimxrt1064, mimxrt1165, mimxrt1166, mimxrt1171,
# mimxrt1172, mimxrt1173, mimxrt1175, mimxrt1176, mimxrt1181, mimxrt1182, mimxrt1187, mimxrt1189, mimxrt533s,
# mimxrt555s, mimxrt595s, mimxrt685s, rw610, rw612>
family: mimxrt1189
# -----------------------------------------===== MCU revision [Optional] =====------------------------------------------
# Description: Revision of silicon. The 'latest' name, means most current revision.
# Possible options: <a0, b0, latest>
revision: latest
# ----------------------------------------===== Peripheral name [Required] =====----------------------------------------
# Description: Name of the external memory support peripheral
# Possible options: <flexspi_nor, flexspi_nand, semc_nor, semc_nand, spi_nor, mmc, sd>
peripheral: flexspi_nor
# ----------------------------------------===== Interface name [Optional] =====-----------------------------------------
# Description: Interface name of some other details about connection
interface: quad_spi
# -------------------------------------===== Option words settings [Required] =====-------------------------------------
# Description: The The option word settings
settings:
  # ----------------------------------------===== ConfigOption0 [Optional] =====----------------------------------------
  # Description: Offset: 0x00000000, Width: 32b; FlexSPI NOR Configuration Option 0
  ConfigOption0:
    # ------------------------------------------===== MaxFreq [Optional] =====------------------------------------------
    # Description: Offset: 0b, Width: 4b, The maximum work frequency for specified Flash device; 0 - Don't change
    # FlexSPI clock setting
    MaxFreq: 0
    # -------------------------------------------===== Misc [Optional] =====--------------------------------------------
    # Description: Offset: 4b, Width: 4b, Specify miscellaneous mode for selected flash type; Experimental feature, do
    # not use in products, keep it as 0.
    # - NotEnabled, (0): Miscellaneous mode is not enabled
    # - Mode0-4-4, (1): Enable 0-4-4 mode for High Random Read performance
    # - SwappedMode, (3):  Data Order Swapped mode (for MXIC OctaFlash only)
    # - InternalLoopBack, (5):  Select the FlexSPI data sample source as internal loop back, more details please refer
    # FlexSPI usage
    # - StandSpiMode, (6):  Config the FlexSPI NOR flash running at stand SPI mode
    # Possible options: <NotEnabled, Mode0-4-4, SwappedMode, InternalLoopBack, StandSpiMode>
    Misc: NotEnabled
    # --------------------------------------===== QuadEnableType [Optional] =====---------------------------------------
    # Description: Offset: 8b, Width: 4b, Specify the Quad Enable sequence, only applicable for device that only JESD216
    # compliant, 

#### Once the template is configured for the new memory, it can be used to export a blhost script with **nxpmemcfg blhost-script** like this:

In [22]:
%! nxpmemcfg blhost-script -c "workspace/cfg_opt_templates/ow_flexspi_nor.yaml" -ix 1 -o $CUSTOM_CFG_MEM_FILE --fcb $CUSTOM_FCB_FILE --force

nxpmemcfg blhost-script -c "workspace/cfg_opt_templates/ow_flexspi_nor.yaml" -ix 1 -o workspace/custom_config_mem.bls --fcb workspace/custom_fcb.bin --force 
Exported blhost script.


#### Let's inspect the blhost-script generated from the **flexspi_nor** template (note the difference in the option word):

In [23]:
Code(filename=CUSTOM_CFG_MEM_FILE, language="bash")

# BLHOST configure memory programming script
# Generated by SPSDK NXPMEMCFG tool
# Chip: mimxrt1189
# Peripheral: flexspi_nor
# Instance: 1

# Switch the instance of the peripheral to 1:
fill-memory 0x1FFE0000 4 0xCF900001
configure-memory 9 0x1FFE0000

# Configure memory:
# Option word 0: 0xC0000000
fill-memory 0x1FFE0000 4 0xC0000000
configure-memory 9 0x1FFE0000

# Script to erase FCB location, create FCB and read back a FCB block:
flash-erase-region 0x28000000 0x1000
fill-memory 0x1FFE0000 4 0xF000000F
configure-memory 9 0x1FFE0000
read-memory 0x28000400 0x200 workspace/custom_fcb.bin

## 7. Build a bootable image and load it into a memory configured though the blhost-script.

#### This process is explained in the [Secure Boot notebook](rt118x_secure_boot.ipynb)